# Abstract
#### 몇주에 접속했는지에 대한 패턴을 구해서 이를 feature로 만듦

### 1. 접속 패턴을 기록

### 2. class 별 확률을 할당

# 수정 사항
## Ver1. 8/30 pattern 파익 및 mean-encoding

In [6]:
## 0. 데이터, package load

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  itertools import combinations
sns.set(palette='pastel')
sns.set(font_scale=2)

In [8]:
train_act=pd.read_csv('transformed/train_real_values.csv')
test_act=pd.read_csv('transformed/test_real_datas_int.csv')

In [17]:
labels=pd.read_csv('FE and error_set/error_set_0.741.csv')

## 1. 접속 패턴을 기록하여 activity에 추가
#### 0은 접속한 주차, 1은 접속안한 주차를 나타냄(왼쪽부터 1주~8주)

In [162]:
train_weekly=train_act.pivot('new_id','wk','cnt_dt')
train_weekly[train_weekly.isna()==False]=1
train_weekly=train_weekly.fillna(0)
train_weekly['play_pattern']=train_weekly.apply((lambda x:''.join(x.astype(int).astype(str).tolist())),axis=1)
train_weekly=train_weekly.reindex(index = train_weekly.index.to_series().str.replace('tr','').astype(int).sort_values().index)
train_weekly['new_id']=train_weekly.index
train_weekly.to_csv('Features/train_weekly_playpattern.csv',index=False)
train_weekly.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,play_pattern,new_id
new_id,,,,,,,,,,
tr0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,00000001,tr0
tr1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,00000001,tr1
tr2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,00001101,tr2
tr3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,00000001,tr3
tr4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11111111,tr4


In [163]:
test_weekly=test_act.pivot('new_id','wk','cnt_dt')
test_weekly[test_weekly.isna()==False]=1
test_weekly=test_weekly.fillna(0)
test_weekly['play_pattern']=test_weekly.apply((lambda x:''.join(x.astype(int).astype(str).tolist())),axis=1)
test_weekly=test_weekly.reindex(index = test_weekly.index.to_series().str.replace('te','').astype(int).sort_values().index)
test_weekly['new_id']=test_weekly.index
test_weekly.to_csv('Features/test_weekly_playpattern.csv',index=False)
test_weekly.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,play_pattern,new_id
new_id,,,,,,,,,,
te0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11111111,te0
te1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11111111,te1
te2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,00000101,te2
te3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,00000111,te3
te4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,01111111,te4


## 2. 접속 pattern의 mean encoding

In [137]:
label_pattern=labels.copy()
label_pattern=pd.merge(label_pattern,train_weekly.loc[:,['play_pattern','new_id']])
label_pattern.head()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'new_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,new_id,label,y_pred,play_pattern
0,tr0,month,2month,00000001
1,tr1,week,week,00000001
2,tr2,retained,retained,00001101
3,tr3,week,week,00000001
4,tr4,retained,retained,11111111


In [138]:
# 2.pattern별 평균 확률

In [139]:
#페턴의 라벨별 count를 해당 패턴의 전체 count로 나눠줌
mean_value=pd.DataFrame(label_pattern.groupby('play_pattern')['label'].value_counts()/label_pattern.groupby('play_pattern')['label'].count())
mean_value.head()

label
play_pattern label             
00000001     week      0.328276
             2month    0.313959
             month     0.311313
             retained  0.046452
00000011     week      0.387105

In [140]:
#multiindex여서 조정이 필요함
mean_value['play_pattern']=mean_value.reset_index(level=1, drop=True).index
mean_value['label_probability']=mean_value.index.droplevel()
mean_value.index=range(len(mean_value))
mean_value.columns=['prob','play_pattern','label']
mean_value.index=range(len(mean_value))
mean_value.head()

,prob,play_pattern,label
0,0.328276,00000001,week
1,0.313959,00000001,2month
2,0.311313,00000001,month
3,0.046452,00000001,retained
4,0.387105,00000011,week


In [144]:
#play pattern 별 label에서의 평균 확률을 구해줌
mean_value=mean_value.pivot('play_pattern','label','prob')
mean_value.columns=['pattern_2month','pattern_month','pattern_retained','pattern_week']
mean_value['play_pattern']=mean_value.index
mean_value.head()

,pattern_2month,pattern_month,pattern_retained,pattern_week,play_pattern
play_pattern,,,,,
00000001,0.313959,0.311313,0.046452,0.328276,00000001
00000011,0.217243,0.356454,0.039199,0.387105,00000011
00000101,0.296547,0.399507,0.093711,0.210234,00000101
00000111,0.241644,0.230255,0.163159,0.364942,00000111
00001001,0.348409,0.182580,0.102178,0.366834,00001001


In [155]:
#train_set에 붙여줌
train_mean_encoded=pd.merge(train_weekly,mean_value, on='play_pattern').iloc[:,8:]
train_mean_encoded=train_mean_encoded.reindex(index=train_mean_encoded.new_id.str.replace('tr','').astype(int).sort_values().index)
train_mean_encoded.head()# 인덱스 수정

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'play_pattern' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,play_pattern,new_id,pattern_2month,pattern_month,pattern_retained,pattern_week
0,00000001,tr0,0.313959,0.311313,0.046452,0.328276
1,00000001,tr1,0.313959,0.311313,0.046452,0.328276
25704,00001101,tr2,0.426829,0.081301,0.284553,0.207317
2,00000001,tr3,0.313959,0.311313,0.046452,0.328276
25950,11111111,tr4,0.143659,0.171169,0.616149,0.069024


In [156]:
#test_set에 붙여줌
test_mean_encoded=pd.merge(test_weekly,mean_value, on='play_pattern').iloc[:,8:]
test_mean_encoded=test_mean_encoded.reindex(index=test_mean_encoded.new_id.str.replace('te','').astype(int).sort_values().index)
test_mean_encoded.head()# 인덱스 수정

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'play_pattern' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,play_pattern,new_id,pattern_2month,pattern_month,pattern_retained,pattern_week
0,11111111,te0,0.143659,0.171169,0.616149,0.069024
1,11111111,te1,0.143659,0.171169,0.616149,0.069024
9655,00000101,te2,0.296547,0.399507,0.093711,0.210234
10314,00000111,te3,0.241644,0.230255,0.163159,0.364942
11947,01111111,te4,0.406124,0.275356,0.185626,0.132894


In [159]:
#저장
train_mean_encoded.iloc[:,1:].to_csv('Features/train_playpattern_mean_encoding.csv',index=False)
train_mean_encoded.to_csv('Features/train_playpattern_mean_encoding_with_pattern.csv',index=False)
test_mean_encoded.iloc[:,1:].to_csv('Features/test_playpattern_mean_encoding.csv',index=False)
test_mean_encoded.to_csv('Features/test_playpattern_mean_encoding_with_pattern.csv',index=False)

In [161]:
sum(train_weekly.play_pattern=='00000001')

25704